# Single-Qubit Simulator


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
Quantum simulators can simulate the behavior of real quantum computers with high fidelity, helping users learn quantum computing knowledge, despite the difficulty of accessing quantum computing hardware due to shortage of resources. Additionally, quantum simulator can perform numerical simulations for researchers and developers to help them accelerate quantum computing algorithm development and quantum chip manufacturing. This tutorial describes how to use Quanlse's single-qubit simulator to consider the dephasing noise and amplitude noise and simulate the gate operation and readout process of superconducting single-qubit system. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Define simulator parameters and pulses
- Running the simulator
- Simulation Results
- User case: Pi pulses calibration using Rabi oscillation
- Summary

## Introduction

  At the pulse level, a single-qubit's control and readout process requires applying pulses to the qubit. In this tutorial, the single-qubit simulator enables the simulation of the kinetic evolution of the control and readout process based on the control pulse, readout pulse, and noise parameters input by the user.
  
  The system Hamiltonian of a three-level superconducting qubit in the rotating frame after RWA (Rotating Wave Approximation) can be written as \[1\]:
$$
\hat{H}_{\rm sys}(t) = \hat{H}_{\rm anharm} + \hat{H}_{\rm noise}(t) + \hat{H}_{\rm ctrl}(t).
$$

where the anharmonicity term is related to qubit's anharmonicity parameter $\alpha$:
$$
\hat{H}_{\rm anharm} = \frac{\alpha}{2} \hat{a}^\dagger \hat{a}^\dagger \hat{a}\hat{a}.
$$

Where $\hat{a}^\dagger$ and $\hat{a}$ are the creation and annihilation operators of the three-level qubit, respectively. In this single-qubit simulator, we specifically introduce two noise terms: the dephasing noise $\hat{H}_{\rm deph}$ due to fluctuations of the ambient magnetic field and the amplitude noise $\hat{H}_{\rm amp}$ due to fluctuations of the control pulse waveform \[1\]:
$$
\hat{H}_{\rm noise}(t) = \hat{H}_{\rm deph} + \hat{H}_{\rm amp}(t),
$$

$$
\hat{H}_{\rm deph} = \frac{1}{2}\mu \hat{a}^\dagger \hat{a},
$$

$$
\hat{H}_{\rm amp}(t) = \epsilon \hat{H}_{\rm ctrl}.
$$

Here, the probability distributions of the dephase noise parameter $\mu$ and the amplitude noise parameter $\epsilon$ follow Gaussian and Lorentzian distributions, respectively:
$$
P(\mu) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\mu^2 / 2\sigma^2},
$$

$$
P(\epsilon) = \frac{1}{\gamma\pi(1+\epsilon^2)}.
$$

Therefore, we characterize these two types of noise by the coefficients $\sigma$ and $\gamma$ related to the probability distribution. Users can add the noise of the simulated system by entering these two parameters.

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import 1-qubit noisy simulator at the pulse level
from Quanlse.remoteSimulator import remoteNoisySimulator as noisySimulator
from Quanlse.Utils.Plot import plotSched

# Import the tool to design the pulse schedule
from Quanlse.Utils.Waveforms import addPulse

# Import other math tools
from math import pi, exp

# Import tools for visualization
from matplotlib import pyplot as plt

import numpy as np

# Import tool for analysis
from scipy.optimize import curve_fit

To use the Quanlse Cloud Service, we need to acquire a token from http://quantum-hub.baidu.com to get access to the cloud. 

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

## Define simulator parameters and pulses
We first define the number of experimental executions `shots`, the detuning parameter $\alpha$ for the qubit,  and the parameters $\sigma$ and $\gamma$ associated with noise (both default values are 0, i.e., the noiseless case).


In [ ]:
shots = 512  # The number of shots 
anharm = -0.3472 * (2 * pi)  # The anharmonicity of the qubit, GHz
deph_sigma = 0.01120487  # Dephasing error
amp_gamma = 0.0159529  # Amplitude (over-rotation) error

Then we use `addPulse` to define the pulse sequence. This function passes the strings `x` and `y` into the parameter `channel` to define the channel (X or Y channel) to which the pulse is applied; the parameters `t0` and `tg` are the start time and duration of the pulse waveform, respectively; and `f` represents the pulse waveform that the user wants to apply.

The followings are three examples showing how we can define a pulse waveform in the simulator using `addPulse()`. Before we start, we need to create an empty list to initialize the pulse sequence:

In [ ]:
ctrlSeq = []  # Initialize the pusle sequence by creating an empty list 

- **Using preset waveform functions:**
Users can define the pulse by using the waveform preset by Quanlse and the input waveform parameters.

In [ ]:
tg0 = 20  # The duration of the pulse
para0 = {'a': 0.2, 'tau': tg0 / 2, 'sigma': tg0 / 8}  # The parameters of the gaussian waveform
ctrlSeq.append(addPulse(channel='x', t0=0, t=tg0, f='gaussian', para=para0))  # Add this pulse to the pulse sequence

- **Using user-defined waveform functions:**
Users can also customize the waveform function in the form of `func(t, args)`, where the parameter `t` is the pulse duration.

In [ ]:
def func(t, args):
    a, tau, sigma = args["a"], args["tau"], args["sigma"]
    if sigma == 0:
        return 0
    return a * exp(- ((t - tau) ** 2 / (2 * sigma ** 2)))

and add it to the pulse sequence:

In [ ]:
tg1 = 30
para1 = {'a': 0.2, 'tau': tg1 / 2, 'sigma': tg1 / 8}
ctrlSeq.append(addPulse(channel='y', t0=tg0, t=tg1, f=func, para=para1))

- **Using a user-defined waveform sequence:**
Users can also enter a list describing the pulse amplitude directly into `seq`.

In [ ]:
tg2 = 40
para2 = {'a': 0.3, 'tau': tg2 / 2, 'sigma': tg2 / 8}
seq = [func(t, para2) for t in range(tg2)]
ctrlSeq.append(addPulse(channel='x', t0=tg0+tg1, t=tg2, seq=seq))

In addition, the readout process also requires us to define the readout pulse, using the function `addPulse()`, while the `channel` is correspondingly changed to `readout`. Here, users need to pay attention that the readout pulse must be added after the end of all control operations, i.e., the start time of the readout pulse `t0` should be bigger than the end time of the control pulse:

In [ ]:
roSeq = addPulse(channel='readout', t0=tg0+tg1+tg2, t=400, f='square', para={'a': 0.8})

We can use the function  `plotSched()` to visualize the user-defined pulse sequence. If the readout pulse is not passed, then the readout pulse is not displayed. 

In [ ]:
plotSched(ctrlSeq)

To display the readout pulse sequence, run the code in the cell below by using ``plotSched``：

In [ ]:
plotSched(waveDataCtrl=ctrlSeq, waveDataReadout=roSeq)

## Running the simulator

After the above preparations are done, we can pass the defined pulse sequence into the simulator `noisySimulator()` (if no readout pulse is passed, there is no readout simulation process). We first simulate without noise, i.e., without passing in the noise parameters $\sigma$ and $\gamma$, while printing to see the control pulse sequence：

In [ ]:
res0 = noisySimulator(waveDataCtrl=ctrlSeq, waveDataReadout=roSeq, shots=shots)

It is also possible to pass in the previously defined noise parameters to run simulations with noise introduced:

In [ ]:
res1 = noisySimulator(dephSigma=deph_sigma, ampGamma=amp_gamma, waveDataCtrl=ctrlSeq, waveDataReadout=roSeq, shots=shots)

## Result

The function `noisySimulator` returns a dictionary of results with the following keys:
- 'prob0': the probability that the result is '0' without the readout simulation.
- 'prob1': the probability that the result is '1' without the readout simulation.
- 'iq_data': the IQ data result of the readout simulation.
- 'counts': the result of the counts after readout simulation.

That is, we can view the readout results in different forms:

- **0-1 count results:** 
We can obtain the population of the results both without and with readout simulations to view the effect of readout noise.

In [ ]:
label = ['0', '1']
x = np.arange(len(label))
# without noise term and readout simulation
y1 = [res0['prob0'], res0['prob1']]
# without noise term but with readout simulation
y2 = [res0['counts']['0']/shots, res0['counts']['1']/shots]
width = 0.35

fig, ax = plt.subplots()

plt.bar(x - width / 2, y1, width=width, color='blue', label='without readout simulation')

plt.bar(x + width / 2, y2, width=width, color='red', label='with readout simulation')

for i, v in enumerate(y1):
    plt.text(x[i] - width / 2, v + 0.01, str(round(v, 3)))

for i, v in enumerate(y2):
    plt.text(x[i] + width / 2, v + 0.01, str(round(v, 3)))

ax.set_ylabel('population')
ax.set_title('0-1 counts without noise term')
ax.set_xticks(x)
ax.set_xticklabels(label)
ax.legend()

plt.show()

Similarly, it is possible to view the resulting population numbers for both noiseless and noisy simulations.

In [ ]:
label = ['0', '1']
x = np.arange(len(label))
# without noise term but with readout simulation
y1 = [res0['counts']['0']/shots, res0['counts']['1']/shots]
# with noise term and readout simulation
y2 = [res1['counts']['0']/shots, res1['counts']['1']/shots]

# plot the population results
width = 0.35

fig, ax = plt.subplots()

plt.bar(x - width / 2, y1, width=width, color='blue', label='without noise term')

plt.bar(x + width / 2, y2, width=width, color='red', label='with noise term')

for i, v in enumerate(y1):
    plt.text(x[i] - width / 2, v + 0.01, str(round(v, 3)))

for i, v in enumerate(y2):
    plt.text(x[i] + width / 2, v + 0.01, str(round(v, 3)))

ax.set_ylabel('population')
ax.set_title('0-1 counts with readout simulation')
ax.set_xticks(x)
ax.set_xticklabels(label)
ax.legend()

plt.show()

**IQ distribution data results:**

When a readout pulse is passed to the parameter `waveDataReadout` of the function `noisySimulator()`, the data returned by the function already includes the distribution of the readout results in phase space. The user can obtain the IQ distribution by processing the key values corresponding to the key `iq_data` of the returned result dictionary \[2\].

In [ ]:
plt.scatter(np.array(res1['iq_data']['0'])[:, 0], np.array(res1['iq_data']['0'])[:, 1], marker='.', label='0')
plt.scatter(np.array(res1['iq_data']['1'])[:, 0], np.array(res1['iq_data']['1'])[:, 1], marker='.', label='1')
plt.xlabel('$I$')
plt.ylabel('$Q$')
plt.legend()
plt.show()

In the above figure, each dot on the plane represents a certain readout result, and the number of dots is the number of executions `shots`. The larger the overlap in the distribution of the different colored dots, the worse the readout results.

Below, we use an example further to understand the usage scenario of this single-qubit simulator.

## User case: Pi pulses calibration using Rabi oscillation

In this section, we perform calibration experiments of Pi pulses using Rabi oscillations, i.e., fixing the pulse duration, varying the pulse amplitude, and obtaining the population number of the $|1\rangle$ state. We use this single-qubit simulator to model this process.

First, we define the range of pulse amplitudes `ampList` and the number of samples `num` for the experiment and the values of the other pulse parameters described in the previous section of the tutorial.

Note: The simulation will take approximately 3-5 minutes. If you want to reduce the simulation's running time, you can reduce the number of runs `shots` and the number of samples `num`, but the simulation results will also be worse.

In [ ]:
shots = 512  # The number of shots 
anharm = -0.3472 * (2 * pi)  # The anharmonicity of the qubit, GHz
deph_sigma = 0.01120487  # Dephasing error
amp_gamma = 0.0159529  # Amplitude (over-rotation) error
num = 40  # The number of sampling points
ampList = np.linspace(0, 0.4, num)  # The list of the amplitudes 
tg_ro = 100  # The duration of the readout pulse
tg_ctrl = 60  # The duration of the control pulse

Then we use the `for` loop to simulate different pulse amplitudes and record the $|1\rangle$ state population both without and with readout simulation for each different amplitude.

In [ ]:
# Add readout simulation channel 
waveDataReadout = addPulse(channel='readout', t0=tg_ctrl, t=tg_ro, f='square', para={'a': 0.8})

prob1List = []
counts1List = []
count = 1

for amp in ampList:
    print(f'running data for amp: {amp}, count: {count} in {num}\n')
    count += 1
    para = {'a': amp, 'tau': tg_ctrl / 2, 'sigma': tg_ctrl / 8}
    waveData = addPulse(channel='x', t0=0, t=tg_ctrl, f='gaussian', para=para)
    res = noisySimulator(waveDataCtrl=waveData, waveDataReadout=waveDataReadout, anharm=anharm, dephSigma=deph_sigma, ampGamma=amp_gamma, shots=shots)
    prob1List.append(res['prob1'])
    counts1List.append(res['counts']['1'])

The results of Rabi oscillations with and without readout simulations are plotted separately: 

In [ ]:
y1 = prob1List
y2 = (np.array(counts1List) / shots).tolist()

plt.plot(ampList, prob1List, '.b', label='without readout simulation')
plt.plot(ampList, np.array(counts1List) / shots, '.r', label='with readout simulation')
plt.xlabel('Amplitude')
plt.ylabel('Probability of being in |1>')
plt.title('Rabi Oscillation')
plt.legend(loc='upper right')

When controlling a single qubit, the Pi pulse (that is, the $\pi$ pulse) corresponds to the operation of rotating the qubit around the X-axis by an angle of 180 degrees on the Bloch sphere. Here, since our qubit's initial state is $|0\rangle$ state by default, this operation can transform our qubit into $|1\rangle$ state under ideal conditions. So we can use the Rabi oscillation experiment to find the pulse amplitude corresponding to such operation by getting the Rabi oscillation curve shown in the graph above, that is, the Pi pulse amplitude described in the tutorial. We can see that when the amplitude is 0, qubit is in the ground state $|0\rangle$; when the amplitude is about 0.17, there is approximately $100\%$ probability that the qubit is flipped into $|1\rangle$ state. Thus we can know: in this single-qubit system we defined, the Pi pulse amplitude is about 0.17.

Next, we fit the obtained curve to obtain a more accurate Pi pulse amplitude, here we choose to use the cosine function $y = -a\cos{(bx)} + 0.49$ as the fitting function. For different hardware parameters and sampling intervals, users can also customize the corresponding fitting function.

In [ ]:
# define the fitting curve
def fit(x, a, b):
    return -a * np.cos(b * x) + 0.49

We fit the above Rabi oscillogram using the `scipy.curve_fit` function to find the Pi pulse amplitude simulated by the simulator.

In [ ]:
paraFit, cov = curve_fit(fit, ampList, y2)
step = 0.01
y3 = [fit(x, paraFit[0], paraFit[1]) for x in np.arange(ampList[0], ampList[-1], step)]

After the fit is finished, we can draw a graph to see the fit:

In [ ]:
plt.plot(np.arange(ampList[0], ampList[-1], step), y3, label='fitting')
plt.plot(ampList, y2, '.r', label='with readout simulation')
plt.xlabel('Amplitude')
plt.ylabel('Probability of being in |1>')
plt.title('Rabi Oscillation')
plt.legend(loc='upper right')
plt.show()

We see that the fit function effectively fits the simulation results with the readout process, and thus the calibrated Pi pulse amplitude can be obtained as follows:

In [ ]:
piAmp = 0 + step * y3.index(max(y3))
print(f'Pi pulse amplitude: {piAmp}')

In the experiment, we can use the data from the calibrated Pi pulses as a basis for other advanced superconducting quantum control experiments.

## Summary

This tutorial describes how to simulate a superconducting single-qubit measurement and readout process considering partial noise using a quantum pulse and visualize the results. Users can click on this link [tutorial-1qubit-simulator.ipynb](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-1qubit-simulator.ipynb) to jump to the corresponding GitHub page for this Jupyter Notebook documentation to get the relevant code, try the different parameter values for further exploring the function of the Quanlse Simulator module.

## Reference

\[1\] [Carvalho, Andre RR, et al. "Error-robust quantum logic optimization using a cloud quantum computer interface." *arXiv preprint arXiv:2010.08057* (2020).](https://arxiv.org/abs/2010.08057)

\[2\] [Blais, Alexandre, et al. "Circuit quantum electrodynamics." *arXiv preprint arXiv:2005.12667* (2020).](https://arxiv.org/abs/2005.12667)